In [1]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
from datetime import datetime, timedelta
import datetime 
import json
from pymongo import MongoClient
import pprint
import csv
hostip='192.168.1.105' #host ip

In [2]:
client = MongoClient(hostip, 27017) # connect to
database_names = client.list_database_names()
for database_name in database_names:
    print(database_name)

admin
config
fit3182
fit3182DD
fit3182_db
local


In [3]:
db = client.fit3182
climate_db= db["climate_db"]


In [4]:

def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8') # encode as utf-8
        producer_instance.send(topic_name, key=key_bytes, value=data)  # encode as utf-8 and send the message 
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'], # send to a specific server
                                  value_serializer=lambda x:dumps(x).encode('ascii'), # for encoding as  ascii
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    

In [5]:
def collectHotSpot(filename,n): # function to send hotspot data
  
    
    with open(filename, 'r') as csvfile:
        #convert csv to dict 
        data = csv.DictReader(csvfile)
        prep_data = []
        for row in data:
            
            row['publisher']=n # publisher 
            row['published_time']= datetime.datetime.now().time().strftime("%H:%M:%S") # to get current time
           
            prep_data.append(row)
             
        return prep_data

In [6]:

t=2
if __name__ == '__main__':
    topic = 'AQUA_HotSpot' # streaming for the AQUA Hotspot
    
    print('Publishing records..')
    producer = connect_kafka_producer() # initialise kafka
    terHPSet = collectHotSpot('hotspot_AQUA_streaming.csv',2) #read from aqua satellite 
   
    random.seed(32939375)
    while True:
        data = terHPSet[(random.randrange(0,len(terHPSet)))] # choose a random data
        
        
        publish_message(producer, topic, 'jsondata', data) #send message
        sleep(t)

Publishing records..
Message published successfully. Data: {'latitude': '-36.4721', 'longitude': '142.2586', 'confidence': '88', 'surface_temperature_celcius': '64', 'publisher': 2, 'published_time': '13:35:59'}
Message published successfully. Data: {'latitude': '-36.6756', 'longitude': '141.708', 'confidence': '63', 'surface_temperature_celcius': '57', 'publisher': 2, 'published_time': '13:35:59'}
Message published successfully. Data: {'latitude': '-36.9664', 'longitude': '142.1561', 'confidence': '81', 'surface_temperature_celcius': '55', 'publisher': 2, 'published_time': '13:35:59'}
Message published successfully. Data: {'latitude': '-36.4439', 'longitude': '145.8846', 'confidence': '56', 'surface_temperature_celcius': '39', 'publisher': 2, 'published_time': '13:35:59'}
Message published successfully. Data: {'latitude': '-36.1687', 'longitude': '145.8275', 'confidence': '68', 'surface_temperature_celcius': '44', 'publisher': 2, 'published_time': '13:35:59'}
Message published success

KeyboardInterrupt: 